In [1]:
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
import pandas as pd
import re
import numpy as np
!pip install datasets --quiet
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import seaborn as sns

2024-05-29 13:28:33.689912: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-29 13:28:33.689969: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-29 13:28:33.691475: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
!pip install accelerate
!pip install bitsandbytes
!pip install langchain
!pip install -U trl 
!pip install git+https://github.com/huggingface/peft.git  -U 

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-ss276fdu
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-ss276fdu
  Resolved https://github.com/huggingface/peft.git to commit e7b75070c72a88f0f7926cc6872858a2c5f0090d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import torch
# from langchain import PromptTemplate, HuggingFacePipeline
# from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
from peft import PeftModel, PeftConfig, LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import HfArgumentParser,TrainingArguments,pipeline, logging 
import os,torch, wandb
from datasets import load_dataset
from trl import SFTTrainer

In [4]:
!wget -O train.tsv "https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_train.tsv"

--2024-05-29 13:30:08--  https://raw.githubusercontent.com/neilrs123/Lexical-Complexity-Prediction/master/Dataset/Sub-task%201/lcp_single_train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1746979 (1.7M) [text/plain]
Saving to: 'train.tsv'

train.tsv           100%[===================>]   1.67M  --.-KB/s    in 0.05s   

2024-05-29 13:30:08 (33.7 MB/s) - 'train.tsv' saved [1746979/1746979]



In [5]:
train = pd.read_table("/kaggle/working/train.tsv")
train = train[train["corpus"] == "bible"].drop(["corpus", "id"], axis = 1)

In [6]:
train.head()

,sentence,token,complexity
0,"Behold, there came up out of the river seven c...",river,0.000000
1,I am a fellow bondservant with you and with yo...,brothers,0.000000
2,"The man, the lord of the land, said to us, 'By...",brothers,0.050000
3,Shimei had sixteen sons and six daughters; but...,brothers,0.150000
4,He has put my brothers far from me.\tbrothers\...,sons,0.055556


In [7]:
train = train.sample(frac = 1)
train = train[:-200]

In [8]:
train

,sentence,token,complexity
23,"Yet will I leave seven thousand in Israel, all...",knees,0.000000
1501,"They came to Hilkiah the high priest, and deli...",threshold,0.428571
706,"The wind shall feed all your shepherds, and yo...",shepherds,0.171053
1409,"Then they said to one another, ""Let's not tear...",Scripture,0.375000
148,and you shall offer for the drink offering hal...,drink,0.075000
...,...,...,...
1948,They couldn't strengthen the foot of their mast.,mast,0.430556
1685,"For out of Jerusalem a remnant will go out, an...",Zion,0.333333
1188,"Before I had finished speaking in my heart, be...",shoulder,0.233333
1130,"Come, curse Jacob for me.",Jacob,0.303571


In [9]:
from huggingface_hub import login
login(token = "")

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [10]:
MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.2"

In [11]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

(True, True)

In [12]:
def check_length(row):
    prompt = f"""[INST]Estimate the lexical complexity of the token in the context, give one single float value between 0 and 1 
    where 0 means the lowest lexical complexity and 1 means the highest lexical complexity (very complex). 
    The third quartile at 0.371 meaning most of the tokens complexity below 0.371.
    Use all possible features to accurately estimate it:
    The Context: {row['sentence']}
    Token: {row['token']}[/INST]"""
    inputs = tokenizer(prompt, return_tensors="pt")
    return len(inputs["input_ids"].squeeze()) <= 200

train['is_within_limit'] = train.apply(check_length, axis=1)
filtered_train = train[train['is_within_limit']]

In [13]:
filtered_train

,sentence,token,complexity,is_within_limit
23,"Yet will I leave seven thousand in Israel, all...",knees,0.000000,True
1501,"They came to Hilkiah the high priest, and deli...",threshold,0.428571,True
706,"The wind shall feed all your shepherds, and yo...",shepherds,0.171053,True
1409,"Then they said to one another, ""Let's not tear...",Scripture,0.375000,True
148,and you shall offer for the drink offering hal...,drink,0.075000,True
...,...,...,...,...
1948,They couldn't strengthen the foot of their mast.,mast,0.430556,True
1685,"For out of Jerusalem a remnant will go out, an...",Zion,0.333333,True
1188,"Before I had finished speaking in my heart, be...",shoulder,0.233333,True
1130,"Come, curse Jacob for me.",Jacob,0.303571,True


In [14]:
# Define maximum sequence length
MAX_LENGTH = 200

def format_example(example):
    prompt = f"""[INST]Estimate the lexical complexity of the token in the context, give one single float value between 0 and 1 
    where 0 means the lowest lexical complexity and 1 means the highest lexical complexity (very complex). 
    The third quartile at 0.371 meaning most of the tokens complexity below 0.371.
    Use all possible features to accurately estimate it:
    The Context: {example['sentence']}
    Token: {example['token']}[/INST]"""
    
    # Ensure the complexity value is converted to a torch tensor and wrapped as a dictionary with 'text' and 'labels'.
    return {"text": prompt, "labels": torch.tensor([example["complexity"]], dtype=torch.float)}


In [15]:
formatted_train = filtered_train.apply(format_example, axis=1)

In [16]:
# Prepare dataset
train_dataset = Dataset.from_dict({
    "text": [x["text"] for x in formatted_train],
    "labels": [x["labels"] for x in formatted_train]
})

In [17]:
# # Load base model(Mistral 7B)
# bnb_config = BitsAndBytesConfig(  
#     load_in_4bit= True,
#     bnb_4bit_quant_type= "nf4",
#     bnb_4bit_compute_dtype= torch.bfloat16,
#     bnb_4bit_use_double_quant= False,
# )

bnb_config = BitsAndBytesConfig(  
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.float16,
    bnb_4bit_use_double_quant= True,
llm_int8_enable_fp32_cpu_offload= True)

model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="balanced",
        trust_remote_code=True,
)

model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [18]:
from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [19]:
from transformers import TrainerCallback

class SaveOnLossDecreaseCallback(TrainerCallback):
    def __init__(self):
        super().__init__()
        self.lowest_loss = float('inf')

    def on_log(self, args, state, control, logs=None, **kwargs):
        # Check if 'loss' is in logs and if we're at a logging step
        if logs is not None and 'loss' in logs:
            current_loss = logs['loss']
            # Save the model if the current loss is lower than the lowest recorded loss
            if current_loss < self.lowest_loss:
                self.lowest_loss = current_loss
                # Set save_model to True to trigger saving
                control.should_save = True
                print(f"New lowest loss {current_loss} detected, triggering checkpoint save.")
            else:
                # Avoid saving if the loss did not decrease
                control.should_save = False

callback = SaveOnLossDecreaseCallback()

In [26]:
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=30,
    logging_steps=30,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb",           # Ensure WandB is set up for logging
)

In [27]:
def extract_complexity_score(text):
  match = re.search(r"(\d+(\.\d+)?)", text)
  if match:
    return float(match.group(1))
  else:
    return None # Handle cases where the model doesn't output a number

from sklearn.metrics import mean_squared_error

def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens = True)
  extracted_scores = [extract_complexity_score(text) for text in decoded_preds]
  mse = mean_squared_error(labels, extracted_scores)
  torch.cuda.empty_cache()
  return {"mse": mse} 

In [28]:
# Setting sft parameters
trainer = SFTTrainer(
    model = model,
    args = training_arguments,
    train_dataset = train_dataset,
    peft_config = peft_config,
    dataset_text_field = 'text',
    tokenizer = tokenizer,
    max_seq_length = 200,
    callbacks=[callback],
    compute_metrics = compute_metrics,  
    packing = False
)

Map:   0%|          | 0/1872 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [29]:
# Fine-tune the model
train_results = trainer.train()

# Save the fine-tuned model
trainer.save_model("./fine_tuned_mistral")

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
30,0.188400
60,0.155900
90,0.139400
120,0.155600
150,0.130000
180,0.166400
210,0.149500
240,0.134700
270,0.146100
300,0.125600


New lowest loss 0.1884 detected, triggering checkpoint save.


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


New lowest loss 0.1559 detected, triggering checkpoint save.


...

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


New lowest loss 0.0997 detected, triggering checkpoint save.


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


New lowest loss 0.0852 detected, triggering checkpoint save.


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


In [ ]:
import shutil
shutil.rmtree("/kaggle/working/results")